In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time

In [2]:
# load dataset and labels
X_train = pickle.load(open('pickle_data/X_train.pickle','rb'))
y_train = pickle.load(open('pickle_data/y_train.pickle','rb'))

# load validation dataset and labels
X_validation = pickle.load(open('pickle_data/X_validation.pickle', 'rb'))
y_validation = pickle.load(open('pickle_data/y_validation.pickle', 'rb'))

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_validation.shape =', X_validation.shape)
print('y_validation.shape =', y_validation.shape)

X_train.shape = (1488, 50, 50, 1)
y_train.shape = (1488,)
X_validation.shape = (166, 50, 50, 1)
y_validation.shape = (166,)


In [3]:
# normalize pixel data
X_train = X_train/255.0
X_validation = X_validation/255.0

In [4]:
# define hyperparameters
conv_layer = 2
layer_size = 64
dense_layer = 0

In [6]:
# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
# create a new log of the model to analyze results
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# define model
model = Sequential()

# 1st convolution
model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional conv layers
for _ in range(conv_layer-1):
    # convolution
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    # Max-Pool to reduce tensor size while retaining distict features
    model.add(MaxPooling2D(pool_size=(2,2)))

# flatten model for fully connected layers
model.add(Flatten())

# add dense layers
for _ in range(dense_layer):
    # fully-connected
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# final fully-connected layer (151 matches number of Pokemon categories)
model.add(Dense(151))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])


Train on 1190 samples, validate on 298 samples
Epoch 1/20
1190/1190 [==============================] - 4s 3ms/sample - loss: 1.7132 - accuracy: 0.3042 - val_loss: 1.3989 - val_accuracy: 0.4195
Epoch 2/20
1190/1190 [==============================] - 3s 3ms/sample - loss: 1.3449 - accuracy: 0.4563 - val_loss: 1.2205 - val_accuracy: 0.5268
Epoch 3/20
1190/1190 [==============================] - 3s 3ms/sample - loss: 1.1846 - accuracy: 0.5319 - val_loss: 1.0965 - val_accuracy: 0.5805
Epoch 4/20
1190/1190 [==============================] - 3s 3ms/sample - loss: 1.0329 - accuracy: 0.6227 - val_loss: 1.0967 - val_accuracy: 0.5470
Epoch 5/20
1190/1190 [==============================] - 3s 3ms/sample - loss: 0.9043 - accuracy: 0.6773 - val_loss: 1.0213 - val_accuracy: 0.6007
Epoch 6/20
1190/1190 [==============================] - 3s 3ms/sample - loss: 0.7747 - accuracy: 0.7176 - val_loss: 0.9699 - val_accuracy: 0.6174
Epoch 7/20
1190/1190 [==============================] - 3s 3ms/sample - loss:

In [7]:
# evaluate the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 66.27%


In [8]:
# save the model
model.save('models/' + NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/2-conv-64-nodes-0-dense-1573176707/assets
